# Random Walks en Multiplex de Aerolíneas

## Directorio

In [1]:
#Establecer directorio
cd("$(homedir())/Documents/UNAM/Tesis_Lic")

## Paquetes

Se importan todos los paquetes que se necesitarán para las funciones y visualizaciones del proyecto.

In [1]:
using StatsBase

#using NamedArrays
#using CSV
#using LightGraphs
#using DataFrames

┌ Info: Recompiling stale cache file /Users/yuriko/.julia/compiled/v1.1/CSV/HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1184


In [ ]:
using DelimitedFiles

In [7]:
using RandomNumbers.Xorshifts

In [ ]:
using Plots

## Funciones

### Importar datos

In [ ]:
"""
    importar_matriz(file::string)

Función que importa el archivo de la matriz de adyacencia de R.
"""
function importar_matriz(arch::AbstractString)
    archivo_R = readdlm(arch, ',') 
    archivo_R = archivo_R[2:end, :]
    matriz = archivo_R[:, 2:end]
    nombre = convert(Array{Int64,2}, matriz)
    return nombre
end

### Nombres de datos

In [ ]:
"""
    data_names(M)

Función que obtiene un arreglo con los nombres de aeropuertos de la matriz
"""
function data_names(arch::AbstractString)
    archivo_R = readdlm(arch, ',')
    columna = archivo_R[1, :]
    nombres = columna[2:end]
    return nombres
end

### Grado

In [3]:
"""

    n_in_degree(M::Array{Int64,2}, n::Int64)

Función que obtiene el grado de entrada de un cierto nodo n de la red representada por la matriz de adyacencia M.

Parámetros:
M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}
n nodo a evaluar de tipo Int64

Resultado:
Grado de entrada del nodo n representado por un valor numérico de tipo Int64

Implementación

#### Ejemplos
"""
function n_in_degree(M::Array{Int64,2}, n::Int64)
    grado_entrada = sum(M[n, :])
end

n_in_degree (generic function with 1 method)

In [4]:
"""
in_degree(M)

Función que obtiene el grado de entrada de una red representada por la matriz de adyacencia M.

Parámetros:
* M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}

Resultado:
Devuelve un arreglo de tipo Array{Int64,2} que contiene los grados de entrada de todos los nodos n de la red.

## Implementación

### Ejemplos
"""
function in_degree(M::Array)
    grados_entrada = sum(M, dims = 2)
end

in_degree (generic function with 1 method)

In [5]:
"""
    n_out_degree(M, n)

Función que obtiene el grado de salida de un cierto nodo n de la red representada por la matriz de adyacencia M.

Parámetros:
* M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}
* n nodo a evaluar de tipo Int64

Resultado:
Grado de salida del nodo n representado por un valor numérico de tipo Int64

## Implementación

### Ejemplos
"""

function n_out_degree(M::Array{Int64,2}, n::Int64)
    grado_salida = sum(M[:, n])
end

n_out_degree (generic function with 1 method)

In [6]:
"""
    out_degree(M)

Función que obtiene el grado de salida de una red representada por la matriz de adyacencia M.

Parámetros:
* M matriz de adyacencia que acepta un arreglo de tipo Array{Int64,2}

Resultado:
Devuelve un arreglo de tipo Array{Int64,2} que contiene los grados de salida de todos los nodos n de la red.

## Implementación

### Ejemplos
"""
function out_degree(M::Array)
    grados_salida = sum(M, dims = 1)
end

out_degree (generic function with 1 method)

### Distribución de grado

In [ ]:
"""
Función de distribución de grado de entrada
"""
function distrib_in_degree(M::Array)
    x = in_degree(M)
    degree_count = counts(x)
    return degree_count
end

In [ ]:
"""
Función de distribución de grado de salida
"""
function distrib_out_degree(M::Array)
    x = out_degree(M)
    degree_count = counts(x)
    return degree_count
end

In [ ]:
function plot_distrib(M::Array, tipo::String, titulo::String, color::String)
    
    pyplot()
    
    if tipo == "in"
        distribucion = distrib_in_degree(M::Array)
    elseif tipo == "out"
        distribucion = distrib_out_degree(M::Array)
    end
    distribucion
    #@show distribucion
    graph = bar(distribucion, fillcolor = color, markersize = 3, linealpha = 0.5, 
    title = titulo, 
    xlabel = "Grado", 
    ylabel = "Frecuencia",
    xlims = (-10,300), #obtener límites
    ylims = (-10,1000)) #obtener límites
end

In [ ]:
function plot_distrib(M::Array, tipo::String, titulo::String, color::String)
    
    pyplot()
    
    if tipo == "in"
        distribucion = distrib_in_degree(M::Array)
    elseif tipo == "out"
        distribucion = distrib_out_degree(M::Array)
    end
    distribucion
    #@show distribucion
    graph = bar(distribucion, fillcolor = color, markersize = 3, linealpha = 0.5, 
    title = titulo, 
    xlabel = "Grado", 
    ylabel = "Frecuencia",
    xlims = (-10,300), #obtener límites
    ylims = (-10,1000)) #obtener límites
end

### Vecinos

In [ ]:
"""
    vecinos(M::Array{Int64,2})

Función que obtiene un conjunto de arreglos con los vecinos de cada nodo.
"""
function vecinos(M::Array{Int64,2})
    
    nodos = size(M, 1) #aeropuertos
    vecindades = []
    
    for i in 1:nodos
        vecinos = findall(x -> x != 0, M[i, :])
        push!(vecindades, vecinos)
    end
    vecindades
end

### Random walks

In [8]:
"""

    r_walks(M::Array{Int64,2}, f::Int64, steps::Int64, iter::Int64)

La función r_walks tiene como parámetros:
M, matriz de adyacencia de tipo Array{Int64,2}
f, nodo inicial de tipo Int64
steps, número de pasos de tipo Int64
iter, número de iteraciones de tipo Int64.
La función crea un caminante aleatorio sobre la matriz, el cual tiene igual probabilidad de avanzar 
hacia cualquiera de sus vecinos.
Regresa un arreglo que contiene todas las trayectorias que tomó el caminante aleatorio.
"""
function r_walks(M::Array{Int64,2}, f::Int64, steps::Int64, iter::Int64)
    
    paths = zeros(Int64, (iter, steps + 1))
    s = f
    seed = 1
    vecindades = vecinos(M)

    for i in 1:iter
        paths[i, 1] = f
        
        for p in 1:steps 
            vecindad = vecindades[s]
            
            if vecindad == []
                break
            else
                a = sample(Xoroshiro128Plus(seed), vecindad) 
                paths[i, p + 1] = a
                s = a
                seed += 1
            end
            
        end
        s = f
        
    end
    
    return paths 
end

r_walks (generic function with 1 method)

In [9]:
"""

    random_walk(M::Array{Int64,2}, steps::Int64, iter::Int64)


La función random_walk tiene como parámetros:
M, matriz de adyacencia de tipo Array{Int64,2}
steps, número de pasos de tipo Int64
iter, número de iteraciones de tipo Int64.
La función crea un caminante aleatorio sobre la matriz con igual probabilidad de avanzar 
hacia cualquiera de sus vecinos. Esta función recorre todos los nodos de la red como
nodos iniciales y sobre ellos itera para obtener el número de caminos que se le pidió.
Regresa un arreglo que contiene arreglos asociados a todas las trayectorias que tomó el 
caminante aleatorio para cada nodo inicial de la red.
"""
function random_walk(M::Array{Int64,2}, steps::Int64, iter::Int64) 
    random_walks = []
    nodos = size(M, 1)
    for f in 1:nodos #aeropuertos
        paths = r_walks(M, f, steps, iter)
        push!(random_walks, paths)
    end
    random_walks
end

random_walk (generic function with 1 method)

### Longitud de caminatas

In [12]:
"""
    walk_length(W)
La función walk_length tiene como parámetros:
RW, arreglo de arreglos de tipo Array{Any,1} que corresponde al arreglo de caminatas aleatorias.
La función regresa un arreglo de arreglos que contiene las longitudes de todas las trayectorias que tomó el 
caminante aleatorio para cada nodo inicial de la red.
"""
function walk_length(W)
    
    longitudes = Array[]
    c = 0
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    t = size(W[1], 2) #número de pasos
    longz = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    
    for i in 1:v
        for j in 1:s
            for k in 1:t
                
                if W[i][j, :][k] != 0 
                    c += 1 
                else
                    break
                end
                
            end
            
            longz[j] = c - 1
            c = 0
            
        end
        
        push!(longitudes, longz)
        longz = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    
    end
    return longitudes
end

walk_length (generic function with 1 method)

In [13]:
"""
    walk_av_length(W)
La función walk_av_length tiene como parámetro W, que es un arreglo de arreglos (que se puede obtener de la 
función walk_length). 
Regresa un arreglo con entradas correspondientes al promedio de las longitudes de 
las caminatas aleatorias para cada nodo de inicio antes de caer en un nodo con grado de salida 0.
"""

function walk_av_length(W) #Llamar a función
    
    proms = []
    wlength = walk_length(W)
    
    for i in 1:length(wlength)
        av = mean(wlength[i])
        push!(proms, av)
    end
    proms
end

walk_av_length (generic function with 1 method)

In [ ]:
"""
    walk_layer_length(W)
Función que saca el promedio de longitud de caminata en la capa
"""
function walk_layer_length(W)
    
    x = walk_av_length(W)
    av = mean(x)
    
    return av
end

### Nodos/aeropuertos visitados

In [ ]:
"""
    nodos_visitados(W)
La función caminos tiene como parámetro W que es un arreglo de arreglos.
Regresa los nodos diferentes a los que llega el caminante.
"""
function nodos_visitados(W)
    
    cobert = []
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    t = size(W[1], 2) #número de pasos, no necesario
    #visit = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    visit = []
    
    for i in 1:v        
        for j in 1:s
            u = unique(W[i][j, :])
            push!(visit, u)
        end
        
        push!(cobert, visit)
        visit = []
    end
    
    return cobert
end

### Frecuencia de visitas

In [ ]:
"""
    freq_visitas_nodos(W::Array)

Función que hace un diccionario con frecuencias de visitas
"""
function freq_visitas_nodos(W::Array)
    
    dicc = [] #arreglo vacío
    
    nodos = length(W) #tamaño de caminatas
    iter = size(W[1], 1) #núm de iteraciones
    diccionario = 0
    
    #for i = n
    for i in 1:nodos #corro por todos los nodos
        for j in 1:iter #corro por las iteraciones
            diccionario = countmap(W[i][j, :]) #hago diccionario de freqs de cada caminata
            push!(dicc, diccionario) #guardo el diccionario en un arreglo vacío
        end
        
        diccionario = 0
    end
    return dicc
end

In [ ]:
"""
    distrib_visitas_nodos(W)

Función que crea un arreglo de arreglos con la distribución de frecuencia de visitas
"""
function distrib_visitas_nodos(W)
    
    x = freq_visitas_nodos(W) #diccionario de frecuencias
    l = length(W)
    f = 0
    frec_total = [] #zeros(Array, length(x)) #arreglo de ceros de tamaño núm dicc
    frec_nodo = zeros(Int64, l+1) #arreglo de nodos de tamaño núm nodos para get frec

    for i in 1:length(x) #corro por los nodos

        for j in 1:l+1 #corro por los nodos

            frec_nodo[j] = get(x[i], f, 0) #entrada j de freq nodo la cambio por la entrada j del dicc i
            f += 1
        end
        push!(frec_total, frec_nodo)
        
        f = 0
        frec_nodo = zeros(Int64, l+1)
        
    end
    
    return frec_total
end

In [ ]:
"""
    plot_visit_freq(W)

Función que hace un heatmap de nodos en los que cae, cada renglón corresponde a una caminata
"""
function plot_visit_freq(W)
    
    pyplot()
    x = distrib_visitas_nodos(W)
    h = transpose(hcat(x...))

    xs = [string("n", i) for i = 0:1:length(W)]
    ys = [string("rw", i) for i = 1:length(x)]
    z = h
    heatmap(xs, ys, z, aspect_ratio=1)
end

In [ ]:
"""
    freq_visitas_nodo(W::Array, n)

Función que hace un diccionario con frecuencias para un nodo inicial
"""
function freq_visitas_nodo(W::Array, n)
    
    dicc = [] #arreglo vacío
    
    nodos = length(W) #tamaño de caminatas
    iter = size(W[1], 1) #núm de iteraciones
    diccionario = 0
    
    for j in 1:iter #corro por las iteraciones
          
        diccionario = countmap(W[n][j, :]) #hago diccionario de freqs de cada caminata
          
        push!(dicc, diccionario) #guardo el diccionario en un arreglo vacío
          
    end
    
    return dicc
end

In [ ]:
"""
    distrib_visitas_nodo(W, n)

Función que crea un arreglo de arreglos con la distribución de frecuencias de visitas de un nodo inicial
"""
function distrib_visitas_nodo(W, n)
    
    x = freq_visitas_nodo(W, n) #diccionario de frecuencias
    l = length(W)
    f = 0
    frec_total = [] #zeros(Array, length(x)) #arreglo de ceros de tamaño núm dicc
    frec_nodo = zeros(Int64, l+1) #arreglo de nodos de tamaño núm nodos para get frec

    for i in 1:length(x) #corro por los nodos

        for j in 1:l+1 #corro por los nodos

            frec_nodo[j] = get(x[i], f, 0) #entrada j de freq nodo la cambio por la entrada j del dicc i
            
            
            f += 1
        end
        push!(frec_total, frec_nodo)
        
        f = 0
        frec_nodo = zeros(Int64, l+1)
        
    end
    
    return frec_total
end

In [ ]:
"""
    plot_visit_freq_nodo(W, n)

Función que hace un heatmap de los nodos en los que cae, cada renglón corresponde a una caminata con cierto nodo inicial
"""
function plot_visit_freq_nodo(W, n)
    
    pyplot()
    x = distrib_visitas_nodo(W, n)
    h = transpose(hcat(x...))
    pasos = size(w[1],2)-1

    xs = [string("n", i) for i = 0:1:length(W)]
    ys = [string("rw", i) for i = 1:length(x)]
    z = h
    heatmap(xs, ys, z, aspect_ratio = 1, clims = (0, pasos))
end

In [ ]:
"""
    distrib_nodos_prom(W::Array)

Función que hace una distribución por nodo inicial y muestra todos los nodos iniciales
"""
function distrib_nodos_prom(W::Array)
    
    nodos = length(W) #número de nodos
    dicc = zeros(Float64, nodos, nodos+1) #arreglo ceros
    nombres = 0:1:nodos
    
    for i in 1:nodos #corro por todos los nodos
        conteos = counts(W[i]) #hago diccionario de freqs de cada caminata
        dicc[i,:] = conteos 
    end

    dicc
end

In [ ]:
"""
    plot_freq_nodos_prom(W::Array)

Función que crea un diccionario con frecuencias por nodo inicial y muestra todos
"""
function plot_freq_nodos_prom(W::Array)
    
    pyplot()
    x = distrib_nodos_prom(W)
    
    xs = [string("n", i) for i = 0:1:length(W)]
    ys = [string("rw", i) for i = 1:length(W)]
    z = x
    heatmap(xs, ys, z, aspect_ratio = 1, clims = (0, maximum(x)))
end

### Cobertura

In [15]:
"""
La función cobertura tiene como parámetro W que es un arreglo de arreglos.
Regresa el número de nodos diferentes a los que llega el caminante.
"""
function cobertura(W) 
    
    cobert = []
    
    v = length(W) #tamaño de la matriz, número de nodos
    x = nodos_visitados(W)
    s = size(x[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    longz = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    
    for i in 1:v        
        for j in 1:s
            l = length(x[i][j])
            longz[j] = l
        end
        push!(cobert, longz)
        longz = zeros(Int64, s)
    end
    return cobert
end

cobertura (generic function with 1 method)

In [16]:
"""
    n_cobertura_av(W)
Da el promedio de cobertura la caminata por nodo inicial. 
"""
function n_cobertura_av(W) #Esta es la importante
    
    longitud = []
    x = cobertura(W)
    
    for i in 1:length(x)
        l = mean(x[i])
        push!(longitud, l)
    end
    longitud
end



n_length_av (generic function with 1 method)

In [17]:
"""
    cobertura_av(W)

Da el promedio de la longitud de las caminatas para todos los nodos, como un número.
"""
function cobertura_av(W)
    x = n_cobertura_av(W)
    promedio = mean(x)
    return promedio
end

length_av (generic function with 1 method)

### Máximos y mínimos de longitud

In [18]:
"""
    max_length(W)
Da la longitud máxima de cada iteración de cada nodo inicial.
"""
function max_length(W)
    max = []
    x = cobertura(W)
    for i in 1:length(x)
        m = maximum(x[i])
        push!(max, m)
    end
    max
end

max_length (generic function with 1 method)

In [19]:
"""
    Max_length(W)
Da la longitud máxima total.
"""
function Max_length(W)
    x = max_length(W)
    m = maximum(x)
    return m
end

Max_length (generic function with 1 method)

In [20]:
"""
    av_max_length(W)
Regresa el promedio de la longitud máxima de cada nodo inicial.
"""
function av_max_length(W)
    x = max_length(W)
    prom = mean(x)
    return prom
end

av_max_length (generic function with 1 method)

In [21]:
"""
    min_length(W)
Da la longitud mínima de cada iteración de cada nodo inicial.
"""
function min_length(W)
    min = []
    x = cobertura(W)
    for i in 1:length(x)
        m = minimum(x[i])
        push!(min, m)
    end
    min
end

min_length (generic function with 1 method)

In [22]:
"""
    Min_length(W)
Da la longitud mínima total.
"""
function Min_length(W)
    x = min_length(W)
    m = minimum(x)
    return m
end

Min_length (generic function with 1 method)

In [23]:
"""
    av_min_length(W)
Regresa el promedio de la longitud mínima de cada nodo inicial.
"""
function av_min_length(W)
    x = min_length(W)
    prom = mean(x)
    return prom
end

av_min_length (generic function with 1 method)

### Endpoints

In [ ]:
"""
    endpoints(W)

Regresa un arreglo de arreglos que contiene los nodos en los que cae cada caminata y ya no puede salir.
"""
function endpoints(W) 
    x = nodos_visitados(W)
    atractores = [] #Arreglo vacío que contenga a los atractores
    v = length(W) #tamaño de la matriz, número de nodos
    s = size(W[1], 1) #lo puedo sacar, para todos es el mismo. Es el número de iteraciones
    t = size(W[1], 2) #número de pasos +1
    atract = zeros(Int64, s) #Arreglo de ceros de tamaño número de iteraciones
    
    for i in 1:v        
        for j in 1:s
            if x[i][j, :][end][end] == 0
                at = x[i][j, :][end][end - 1]
                atract[j] = at
            else
                atract[j] = 0
            end
        end
        push!(atractores, atract)
        atract = zeros(Int64, s)
    end
    return atractores
end

In [ ]:
"""
    av_n_endpoints(W)
Función que calcula el promedio de endpoints por nodo
"""
function av_n_endpoints(W) 
    
    x = endpoints(W)
    atractores_prom = [] #Arreglo vacío que contenga a los atractores
    v = length(W) #tamaño de la matriz, número de nodos

    for i in 1:v        
        av = mean(x[i])
        push!(atractores_prom, av)
    end
    atractores_prom
end

### Distribución de frecuencia de endpoints

In [ ]:
"""
    freq_endpoints(W)
Función que da un diccionario con frecuencias
"""
function freq_endpoints(W)
    freqs = []
    x = endpoints(W)
    v = length(W)
    
    for i in 1:v
        frec = countmap(x[i])
        push!(freqs, frec)
    end
    freqs
end

In [ ]:
"""
    distrib_endpoints(W)
Función que saca la distribución de los atractores para toda la capa
"""
function distrib_endpoints(W)
    x = freq_endpoints(W)
    frec_total = zeros(Int64, length(W))
    frec_nodo = zeros(Int64, length(W))

    for i in 1:length(W)
        for j in 1:length(W)
            frec_nodo[j] = get(x[i], j, 0)
        end
        frec_total += frec_nodo
    end
    
    return frec_total
end

## Datos


### Multiplex Global

In [3]:
multiplex = importar_matriz("data/multi_17_du.csv")

3190×3190 Array{Any,2}:
 ""      "AER"   "ASF"   "CEK"   "DME"  …   "BSS"   "AEX"   "GCK"   "MGM"
 "AER"  0       0       0       1          0       0       0       0     
 "ASF"  0       0       0       1          0       0       0       0     
 "CEK"  0       0       0       1          0       0       0       0     
 "DME"  1       1       1       0          0       0       0       0     
 "EGO"  0       0       0       1       …  0       0       0       0     
 "GYD"  0       0       0       1          0       0       0       0     
 "KGD"  0       0       0       1          0       0       0       0     
 "KZN"  1       1       1       1          0       0       0       0     
 "LED"  1       1       1       1          0       0       0       0     
 "MRV"  0       1       0       1       …  0       0       0       0     
 "NBC"  0       0       0       1          0       0       0       0     
 "NJC"  0       0       0       1          0       0       0       0     
 ⋮            

In [4]:
nombres_multiplex = data_names("data/multi_17_du.csv")

## Distribuciones de grado

### Multiplex Global

#### Grado de entrada

In [ ]:
in_degree_multiplex = in_degree(multiplex)

In [ ]:
distrib_in_multiplex = distrib_in_degree(multiplex)

In [ ]:
in_M = plot_distrib(multiplex, "in", "Multiplex Distribución Entrada", "black")

#### Grado de salida

In [ ]:
out_degree_multiplex = out_degree(multiplex)

In [ ]:
distrib_out_multiplex = distrib_out_degree(multiplex)

In [ ]:
out_M = plot_distrib(multiplex, "out", "Multiplex Distribución Salida", "black")

## Vecinos

### Multiplex global

In [ ]:
vecinos_M = @time vecinos(multiplex)

## Caminatas aleatorias

### Multiplex

In [29]:
caminata_M = @time random_walk(multiplex, 2500, 100) #nuevo

 73.894156 seconds (19.17 M allocations: 38.698 GiB, 5.20% gc time)


3189-element Array{Any,1}:
 [1 671 … 1193 1129; 1 1531 … 188 52; … ; 1 837 … 161 1925; 1 4 … 1869 2332] 
 [2 8 … 451 1024; 2 1829 … 1032 135; … ; 2 4 … 60 1761; 2 232 … 0 0]         
 [3 90 … 1710 1600; 3 4 … 1656 1930; … ; 3 478 … 307 1548; 3 90 … 1979 1980] 
 [4 730 … 1870 2386; 4 11 … 1794 886; … ; 4 764 … 938 124; 4 53 … 1078 71]   
 [5 753 … 1158 1175; 5 8 … 580 307; … ; 5 759 … 1015 170; 5 8 … 1391 1081]   
 [6 2282 … 55 1233; 6 451 … 715 217; … ; 6 727 … 1868 358; 6 1561 … 260 336] 
 [7 594 … 398 2930; 7 5 … 726 682; … ; 7 9 … 405 398; 7 1527 … 1385 429]     
 [8 423 … 164 1009; 8 594 … 684 1851; … ; 8 9 … 2385 2829; 8 15 … 357 361]   
 [9 473 … 0 0; 9 2231 … 661 449; … ; 9 220 … 252 1085; 9 724 … 594 429]      
 [10 232 … 715 1546; 10 6 … 438 1226; … ; 10 6 … 428 231; 10 594 … 358 759]  
 [11 4 … 220 1218; 11 724 … 289 264; … ; 11 727 … 1149 1150; 11 727 … 1752 4]
 [12 4 … 1015 2035; 12 594 … 163 560; … ; 12 349 … 338 1110; 12 4 … 305 1012]
 [13 2275 … 1024 168; 13 671 … 1448 1

## Longitud de todas las iteraciones de caminatas 

In [48]:
li_M = @time walk_length(caminata_M)

  1.840936 seconds (5.50 M allocations: 1.365 GiB, 6.08% gc time)


## Longitud promedio de las iteraciones de caminatas de cada nodo inicial

In [31]:
lpn_M = @time walk_av_length(caminata_M)

  1.863774 seconds (5.68 M allocations: 1.374 GiB, 6.06% gc time)


3189-element Array{Any,1}:
 100.0
  96.6
 100.0
 100.0
 100.0
 100.0
 100.0
 100.0
  85.8
 100.0
 100.0
 100.0
 100.0
   ⋮  
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0

In [ ]:
pyplot()
d_lpn_M = scatter(lpn_M, color="#0f87bf",alpha=0.8, 
    title="Multiplex", 
    xlabel="Nodo inicial", 
    ylabel="Promedio long.")

## Longitud promedio de caminatas por capa

In [ ]:
wl_M = @time walk_layer_length(caminata_M)

## Nodos visitados específicamente

In [ ]:
#No lo corro ahora para que no se tarde tanto
#nv_M = @time nodos_visitados(caminata_M)

In [ ]:
#Falta función de aeropuertos específicos

## Frecuencia de visitas

In [ ]:
freq_visit_M = @time freq_visitas_nodos(caminata_M)

In [ ]:
#distrib_visit_M = @time distrib_visitas_nodos(caminata_M)

In [ ]:
#plot_visit_freq(caminata_M)

In [ ]:
#freq_visitas_nodo(W::Array, n)

In [ ]:
#distrib_visitas_nodos(caminata_M, n)

In [ ]:
#plot_visit_freq_nodo(caminata_M, n)

In [ ]:
distrib_prom_M = @time distrib_nodos_prom(caminata_M)

In [ ]:
plot_freq_nodos_prom(caminata_M)

## Cobertura (nodos distintos)

In [36]:
cobi_M = @time cobertura(caminata_M)

  0.160455 seconds (343.94 k allocations: 104.292 MiB, 32.85% gc time)


3189-element Array{Any,1}:
 [79, 77, 74, 77, 86]
 [75, 83, 80, 87, 70]
 [86, 83, 74, 85, 81]
 [85, 76, 88, 88, 78]
 [84, 84, 83, 83, 78]
 [83, 73, 83, 85, 81]
 [82, 79, 78, 85, 81]
 [83, 79, 83, 82, 87]
 [29, 84, 85, 79, 82]
 [77, 87, 78, 80, 80]
 [87, 83, 66, 89, 80]
 [79, 82, 80, 79, 86]
 [83, 86, 83, 87, 85]
 ⋮                   
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     
 [2, 2, 2, 2, 2]     

## Cobertura promedio de las caminatas de cada aeropuerto inicial

In [65]:
cobs_M = @time n_cobertura_av(caminata_M)

  0.000001 seconds (3 allocations: 144 bytes)
  0.286188 seconds (347.15 k allocations: 104.404 MiB, 63.52% gc time)


3189-element Array{Any,1}:
 78.6
 79.0
 81.8
 83.0
 82.4
 81.0
 81.0
 82.8
 71.8
 80.4
 81.0
 81.2
 84.8
  ⋮  
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0
  2.0

In [ ]:
d_cobs_M = scatter(cobs_M, color="#0f87bf",alpha=0.4, 
    title="Long. caminatas por nodo(M)", 
    xlabel="Aeropuertos iniciales", 
    ylabel="Long. prom.")

## Cobertura promedio

In [ ]:
cobp_M = @time cobertura_av(caminata_M)

## Longitudes máximas de caminatas

In [49]:
lmax_M = @time max_length(caminata_M)

  0.128620 seconds (343.96 k allocations: 104.355 MiB, 16.06% gc time)


In [42]:
lMax_M = @time Max_length(caminata_M)

  0.178791 seconds (377.12 k allocations: 106.120 MiB, 15.99% gc time)


96

In [43]:
lavmax_M = @time av_max_length(caminata_M)

  0.135874 seconds (348.09 k allocations: 104.457 MiB, 16.29% gc time)


83.74788334901223

## Longitudes mínimas de caminatas

In [50]:
lmin_M = @time min_length(caminata_M)

  0.285365 seconds (343.96 k allocations: 104.355 MiB, 62.80% gc time)


In [45]:
lMin_M = @time Min_length(caminata_M)

  0.172192 seconds (377.21 k allocations: 106.129 MiB, 16.34% gc time)


2

In [46]:
lavmin_M = @time av_min_length(caminata_M)

  0.293863 seconds (348.06 k allocations: 104.456 MiB, 60.72% gc time)


59.80620884289746

## Endpoints

In [ ]:
endpts_M = @time endpoints(caminata_M)